# 混洗（Shuffling）机制

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("Shuffling").\
    getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

创建一个简单的 RDD：

In [ ]:
import string

rdd = sc.parallelize(string.ascii_uppercase, numSlices=5)
print(rdd.collect())

In [ ]:
rdd.getNumPartitions()

将每个分区的字母合并成一个字符串，从而可以知道分区是如何划分的：

In [ ]:
def concat_letters(iter):
    yield "".join(iter)

rdd.mapPartitions(concat_letters).collect()

将 RDD 重新分区，可以看出来字母的顺序产生了变化：

In [ ]:
rdd2 = rdd.repartition(6)
print(rdd2.collect())

In [ ]:
rdd2.mapPartitions(concat_letters).collect()

使用 `toDebugString()` 查看 RDD 包含的操作：

In [ ]:
print(rdd.toDebugString().decode("UTF-8"))
print("\n\n")
print(rdd2.toDebugString().decode("UTF-8"))

可以看出来 `rdd2` 包含了混洗（shuffling）操作，这也是数据顺序打乱的原因。

如果需要减小分区数目，可以使用 `coalesce()` 函数，避免混洗操作：

In [ ]:
rdd3 = rdd.coalesce(numPartitions=2, shuffle=False)
print(rdd3.collect())
rdd3.mapPartitions(concat_letters).collect()

In [ ]:
print(rdd3.toDebugString().decode("UTF-8"))

如果确实需要增加分区数目，同时希望保持数据顺序，可以在原始 RDD 中增加索引信息：

In [ ]:
print(rdd.zipWithIndex().collect())

In [ ]:
rdd4 = rdd.zipWithIndex().repartition(5)
print(rdd4.collect())

然后定义一个分区映射函数，在合并数据行时获取分区中第一条数据的索引：

In [ ]:
def concat_letters_with_order(iter):
    letters_and_indices = list(iter)
    letters = map(lambda x: x[0], letters_and_indices)
    indices = map(lambda x: x[1], letters_and_indices)
    if len(letters_and_indices) < 1:
        yield ()
    else:
        first_ind = next(indices)
        combined_letters = "".join(letters)
        yield combined_letters, first_ind

rdd5 = rdd4.mapPartitions(concat_letters_with_order)
print(rdd5.collect())

然后过滤空分区，并按生成的索引对 RDD 进行排序：

In [ ]:
rdd5.filter(lambda x: len(x) > 1).collect()

In [ ]:
rdd5.filter(lambda x: len(x) > 1).sortBy(lambda x: x[1], ascending=True).collect()